In [14]:
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [15]:
def load_data(file_name):
    file_path = Path(file_name).resolve()
    excel_file = pd.read_excel(file_path, sheet_name=None)
    sheet_dicts = {
        sheet_name: df.to_dict(orient="records")
        for sheet_name, df in excel_file.items()
    }
    # Assign each to a variable
    vendorA_data = sheet_dicts.get("vendor a")
    vendorB_data = sheet_dicts.get("vendor b")
    vendorC_data = sheet_dicts.get("vendor c")
    vendorD_data = sheet_dicts.get("vendor d")

    return sheet_dicts, vendorA_data, vendorB_data, vendorC_data, vendorD_data

def split_vendor_data(data):
    step_names = [entry['step'] for entry in data]
    
    production_dict = {entry['step']: entry['production'] for entry in data}
    cost_dict = {entry['step']: entry['cost'] for entry in data}
    
    return step_names, production_dict, cost_dict

In [ ]:
def optimize(step_names, budget):
    model = pyo.ConcreteModel(doc="flowsheet Optimization Model")
    def prod_objective(m):

    model.obj = pyo.Objective(rule=prod_objective, sense = pyo.maximize )
    
    def budget_constraint(m):

    model.budget_constraint = pyo.Constraint(rule=budget_constraint)

    opt = pyo.SolverFactory("gurobi")
    opt_success = opt.solve(model)

    model.pprint()


In [17]:
sheet_dicts, vendorA_data, vendorB_data, vendorC_data, vendorD_data = load_data("test-data.xlsx")
print(sheet_dicts)
print(vendorA_data)

{'vendor a': [{'step': 1, 'production': 50, 'cost': 10}, {'step': 2, 'production': 100, 'cost': 25}, {'step': 3, 'production': 75, 'cost': 12}, {'step': 4, 'production': 45, 'cost': 15}], 'vendor b': [{'step': 1, 'production': 50, 'cost': 10}, {'step': 2, 'production': 75, 'cost': 15}, {'step': 3, 'production': 65, 'cost': 12}, {'step': 4, 'production': 45, 'cost': 15}], 'vendor c': [{'step': 1, 'production': 43, 'cost': 21}, {'step': 2, 'production': 100, 'cost': 25}, {'step': 3, 'production': 44, 'cost': 20}, {'step': 4, 'production': 45, 'cost': 30}], 'vendor d': [{'step': 1, 'production': 50, 'cost': 10}, {'step': 2, 'production': 89, 'cost': 25}, {'step': 3, 'production': 57, 'cost': 22}, {'step': 4, 'production': 100, 'cost': 45}]}
[{'step': 1, 'production': 50, 'cost': 10}, {'step': 2, 'production': 100, 'cost': 25}, {'step': 3, 'production': 75, 'cost': 12}, {'step': 4, 'production': 45, 'cost': 15}]
